## Siaf-ETL

The purpose of this notebook is to find a dictionary to solve the differences of the categories written in siaf with those in delitos seguimiento and delitos validados.

In [1]:
from src.utils import conectar_sql
import pandas as pd

conx = conectar_sql()

conectando con big data database....


In [6]:
query1 = """select robos.DELITO_DESAGREGACION,
count(robos.NDD) as TOTAL
from DaaS.robosML robos
group by robos.DELITO_DESAGREGACION
order by count(robos.NDD) desc;"""

desagregation_siaf = pd.read_sql(query1, conx)
desagregation_siaf.shape

(44, 2)

In [7]:
desagregation_siaf.head()

,DELITO_DESAGREGACION,TOTAL
0,ROBO A PERSONAS,419036
1,ROBO A DOMICILIO,127168
2,NO_APLICA,83864
3,ROBO DE MOTOS,76614
4,ROBO DOMICILIOS,72933


In [9]:
query2 = """select robos.delitos_seguimiento_predicted,
count(robos.NDD) as TOTAL
from DaaS.robosML robos
group by robos.delitos_seguimiento_predicted
order by count(robos.NDD) desc;"""

seguimiento = pd.read_sql(query2, conx)
seguimiento.shape

(8, 2)

In [10]:
seguimiento

,delitos_seguimiento_predicted,TOTAL
0,OTROS ROBOS,367002
1,ROBO A PERSONAS,301478
2,ROBO A DOMICILIO,123590
3,"ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEH...",114949
4,ROBO DE MOTOS,84283
5,ROBO A UNIDADES ECONOMICAS,68591
6,ROBO DE CARROS,68218
7,None,5088


In [18]:
seguimiento_set = set(seguimiento.delitos_seguimiento_predicted)
seguimiento_set.discard(None)
# seguimiento_set.discard('OTROS ROBOS')
seguimiento_set

{'OTROS ROBOS',
 'ROBO A DOMICILIO',
 'ROBO A PERSONAS',
 'ROBO A UNIDADES ECONOMICAS',
 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'ROBO DE CARROS',
 'ROBO DE MOTOS'}

In [11]:
query3 = """select robos.delitos_validados_predicted,
count(robos.NDD) as TOTAL
from DaaS.robosML robos
group by robos.delitos_validados_predicted
order by count(robos.NDD) desc;"""

validados = pd.read_sql(query3, conx)
validados.shape

(12, 2)

In [12]:
validados

,delitos_validados_predicted,TOTAL
0,OTROS ROBOS,392949
1,ROBO A PERSONAS,297312
2,ROBO A DOMICILIO,110318
3,"ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEH...",101569
4,ROBO DE MOTOS,86871
5,ROBO DE CARROS,72171
6,ROBO A UNIDADES ECONOMICAS,48739
7,ROBO A INSTITUCIONES EDUCATIVAS,8905
8,ROBO A EMBARCACIONES DE ESPACIOS ACUATICOS,5395
9,None,5088


In [16]:
validados_set = set(validados.delitos_validados_predicted)
validados_set.discard(None)
validados_set

{'OTROS ROBOS',
 'ROBO A DOMICILIO',
 'ROBO A EMBARCACIONES DE ESPACIOS ACUATICOS',
 'ROBO A ESTABLECIMIENTOS DE COLECTIVOS U ORGANIZACIONES SOCIALES',
 'ROBO A INSTITUCIONES EDUCATIVAS',
 'ROBO A PERSONAS',
 'ROBO A UNIDADES ECONOMICAS',
 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'ROBO DE CARROS',
 'ROBO DE MOTOS',
 'ROBO EN INSTITUCIONES PUBLICAS'}

In [24]:
from fuzzywuzzy import process
seguimiento_mapping = {}
for siaf_cat in desagregation_siaf.DELITO_DESAGREGACION.to_list():
    closes_match, score = process.extractOne(siaf_cat, seguimiento_set)
    # print(score)
    if score > 85:
        seguimiento_mapping[siaf_cat] = closes_match
    else:
        seguimiento_mapping[siaf_cat] = 'No match found'

In [25]:
seguimiento_mapping

{'ROBO A PERSONAS': 'ROBO A PERSONAS',
 'ROBO A DOMICILIO': 'ROBO A DOMICILIO',
 'NO_APLICA': 'No match found',
 'ROBO DE MOTOS': 'ROBO DE MOTOS',
 'ROBO DOMICILIOS': 'ROBO A DOMICILIO',
 'ROBO DE CARROS': 'ROBO DE CARROS',
 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS': 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'OTROS ROBOS': 'OTROS ROBOS',
 'ROBO DE ACCESORIOS DE VEHÍCULOS': 'ROBO DE CARROS',
 'ROBO DE VEHÍCULOS': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO DE MOTOCICLETAS': 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'ROBO EN LOCALES COMERCIALES': 'ROBO DE CARROS',
 'ROBO A BIENES DE UNIDADES ECONOMICAS': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO A BIENES DE INSTITUCIONES EDUCATIVAS': 'ROBO DE CARROS',
 'ROBO A BIENES DE INSTITUCION PUBLICA': 'ROBO DE CARROS',
 'ROBO OTROS': 'OTROS ROBOS',
 'ROBO DE BIENES PERSONALES AL INTERIOR DEL VEHÍCULO': 'ROBO DE CARROS',
 'ROBO EN VÍAS O CARRETERAS': 'ROBO DE CARROS',
 'ROBO DE BIENES A EMPRESA/FABRICA': 'ROBO DE CARR

In [ ]:
siaf_seguimiento_dict = {'ROBO A PERSONAS': 'ROBO A PERSONAS',
 'ROBO A DOMICILIO': 'ROBO A DOMICILIO',
 'NO_APLICA': 'No match found',
 'ROBO DE MOTOS': 'ROBO DE MOTOS',
 'ROBO DOMICILIOS': 'ROBO A DOMICILIO',
 'ROBO DE CARROS': 'ROBO DE CARROS',
 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS': 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'OTROS ROBOS': 'OTROS ROBOS',
 'ROBO DE ACCESORIOS DE VEHÍCULOS': 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'ROBO DE VEHÍCULOS': 'ROBO DE CARROS',
 'ROBO DE MOTOCICLETAS': 'ROBO DE MOTOS',
 'ROBO EN LOCALES COMERCIALES': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO A BIENES DE UNIDADES ECONOMICAS': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO A BIENES DE INSTITUCIONES EDUCATIVAS': 'OTROS ROBOS',
 'ROBO A BIENES DE INSTITUCION PUBLICA': 'OTROS ROBOS',
 'ROBO OTROS': 'OTROS ROBOS',
 'ROBO DE BIENES PERSONALES AL INTERIOR DEL VEHÍCULO': 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'ROBO EN VÍAS O CARRETERAS': 'OTROS ROBOS',
 'ROBO DE BIENES A EMPRESA/FABRICA': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO DE BIENES A ENTIDAD PÚBLICA': 'OTROS ROBOS',
 'ROBO DE BIENES A INSTITUCION EDUCATIVA': 'OTROS ROBOS',
 'ROBO A BIENES DE ESTABLECIMIENTOS DE COLECTIVOS U ORGANIZACIONES SOCIALES': 'OTROS ROBOS',
 'ROBO A EMBARCACIONES': 'OTROS ROBOS',
 'ROBO A BIENES DE INSTITUCIONES DE SALUD': 'OTROS ROBOS',
 'ROBO A BIENES DE ENTIDADES FINANCIERAS': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO A VEHICULOS DE TRANSPORTE DE VALORES': 'OTROS ROBOS',
 'ROBO DE BIENES A ENTIDAD PRIVADA': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO DE MOTORES EMBARCACIONES': 'OTROS ROBOS',
 'ROBO A BANCOS - ENTIDADES  FINANCIERAS': 'ROBO A UNIDADES ECONOMICAS',
 'HURTO A PERSONAS': 'No match found',
 'ROBO A  BOTES PESQUEROS – YATES- FIBRAS-VELEROS ETC.': 'OTROS ROBOS',
 'HURTO A DOMICILIO': 'No match found',
 'HURTO A MOTOS': 'No match found',
 'HURTO A BIENES PUBLICOS': 'No match found',
 'HURTO A CARROS': 'No match found',
 'HURTO A ACCESORIOS': 'No match found',
 'HURTO A BIENES DE UNIDADES ECONOMICAS': 'No match found',
 'TRANSITO': 'No match found',
 'ABIGEATO': 'No match found',
 'HURTO A EMBARCACIONES O PARTES EN ESPACIOS ACUATICOS': 'No match found',
 'HURTO A BIENES PATRIMONIALES': 'No match found',
 'HURTO DE LO REQUISADO': 'No match found',
 'ROBO A BIENES PATRIMONIALES': 'OTROS ROBOS',
 'HURTO DE BIENES DE USO POLICIAL O MILITAR': 'No match found'}

In [26]:
validados_mapping = {}
for siaf_cat in desagregation_siaf.DELITO_DESAGREGACION.to_list():
    closes_match, score = process.extractOne(siaf_cat, list(validados_set))
    # print(score)
    if score > 85:
        validados_mapping[siaf_cat] = closes_match
    else:
        validados_mapping[siaf_cat] = 'No match found'

In [27]:
validados_mapping

{'ROBO A PERSONAS': 'ROBO A PERSONAS',
 'ROBO A DOMICILIO': 'ROBO A DOMICILIO',
 'NO_APLICA': 'No match found',
 'ROBO DE MOTOS': 'ROBO DE MOTOS',
 'ROBO DOMICILIOS': 'ROBO A DOMICILIO',
 'ROBO DE CARROS': 'ROBO DE CARROS',
 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS': 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'OTROS ROBOS': 'OTROS ROBOS',
 'ROBO DE ACCESORIOS DE VEHÍCULOS': 'ROBO DE CARROS',
 'ROBO DE VEHÍCULOS': 'ROBO EN INSTITUCIONES PUBLICAS',
 'ROBO DE MOTOCICLETAS': 'ROBO EN INSTITUCIONES PUBLICAS',
 'ROBO EN LOCALES COMERCIALES': 'ROBO DE CARROS',
 'ROBO A BIENES DE UNIDADES ECONOMICAS': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO A BIENES DE INSTITUCIONES EDUCATIVAS': 'ROBO A INSTITUCIONES EDUCATIVAS',
 'ROBO A BIENES DE INSTITUCION PUBLICA': 'ROBO DE CARROS',
 'ROBO OTROS': 'OTROS ROBOS',
 'ROBO DE BIENES PERSONALES AL INTERIOR DEL VEHÍCULO': 'ROBO DE CARROS',
 'ROBO EN VÍAS O CARRETERAS': 'ROBO DE CARROS',
 'ROBO DE BIENES A EMPRESA/FABRICA': 'ROBO DE CARRO

In [ ]:
siaf_validados_dict = {'ROBO A PERSONAS': 'ROBO A PERSONAS',
 'ROBO A DOMICILIO': 'ROBO A DOMICILIO',
 'NO_APLICA': 'No match found',
 'ROBO DE MOTOS': 'ROBO DE MOTOS',
 'ROBO DOMICILIOS': 'ROBO A DOMICILIO',
 'ROBO DE CARROS': 'ROBO DE CARROS',
 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS': 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'OTROS ROBOS': 'OTROS ROBOS',
 'ROBO DE ACCESORIOS DE VEHÍCULOS': 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'ROBO DE VEHÍCULOS': 'ROBO DE CARROS',
 'ROBO DE MOTOCICLETAS': 'ROBO DE MOTOS',
 'ROBO EN LOCALES COMERCIALES': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO A BIENES DE UNIDADES ECONOMICAS': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO A BIENES DE INSTITUCIONES EDUCATIVAS': 'ROBO A INSTITUCIONES EDUCATIVAS',
 'ROBO A BIENES DE INSTITUCION PUBLICA': 'ROBO EN INSTITUCIONES PUBLICAS',
 'ROBO OTROS': 'OTROS ROBOS',
 'ROBO DE BIENES PERSONALES AL INTERIOR DEL VEHÍCULO': 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'ROBO EN VÍAS O CARRETERAS': 'OTROS ROBOS',
 'ROBO DE BIENES A EMPRESA/FABRICA': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO DE BIENES A ENTIDAD PÚBLICA': 'ROBO EN INSTITUCIONES PUBLICAS',
 'ROBO DE BIENES A INSTITUCION EDUCATIVA': 'ROBO A INSTITUCIONES EDUCATIVAS',
 'ROBO A BIENES DE ESTABLECIMIENTOS DE COLECTIVOS U ORGANIZACIONES SOCIALES': 'ROBO A ESTABLECIMIENTOS DE COLECTIVOS U ORGANIZACIONES SOCIALES',
 'ROBO A EMBARCACIONES': 'ROBO A EMBARCACIONES DE ESPACIOS ACUATICOS',
 'ROBO A BIENES DE INSTITUCIONES DE SALUD': 'OTROS ROBOS',
 'ROBO A BIENES DE ENTIDADES FINANCIERAS': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO A VEHICULOS DE TRANSPORTE DE VALORES': 'OTROS ROBOS',
 'ROBO DE BIENES A ENTIDAD PRIVADA': 'ROBO A UNIDADES ECONOMICAS',
 'ROBO DE MOTORES EMBARCACIONES': 'ROBO A EMBARCACIONES DE ESPACIOS ACUATICOS',
 'ROBO A BANCOS - ENTIDADES  FINANCIERAS': 'ROBO A UNIDADES ECONOMICAS',
 'HURTO A PERSONAS': 'No match found',
 'ROBO A  BOTES PESQUEROS – YATES- FIBRAS-VELEROS ETC.': 'ROBO A EMBARCACIONES DE ESPACIOS ACUATICOS',
 'HURTO A DOMICILIO': 'No match found',
 'HURTO A MOTOS': 'No match found',
 'HURTO A BIENES PUBLICOS': 'No match found',
 'HURTO A CARROS': 'No match found',
 'HURTO A ACCESORIOS': 'No match found',
 'HURTO A BIENES DE UNIDADES ECONOMICAS': 'No match found',
 'TRANSITO': 'No match found',
 'ABIGEATO': 'No match found',
 'HURTO A EMBARCACIONES O PARTES EN ESPACIOS ACUATICOS': 'No match found',
 'HURTO A BIENES PATRIMONIALES': 'No match found',
 'HURTO DE LO REQUISADO': 'No match found',
 'ROBO A BIENES PATRIMONIALES': 'OTROS ROBOS',
 'HURTO DE BIENES DE USO POLICIAL O MILITAR': 'No match found'}

In [ ]:
# Now you can replace the original categories with the new ones
# For example, if you have a pandas DataFrame df and the categories are in the 'category' column:
# df['category'] = df['category'].map(category_mapping)